This Jupyter notebook explores relationships between the quantities in the equation

$\displaystyle{T=2\sqrt{\frac{2L}{g}} \int_0^{\theta_{\text{max}}} \frac{d\theta}{\sqrt{\cos\theta-\cos\theta_{\text{max}}}}}$

which governs the period ($T$), length ($L$), and maximum swing of a pendulum ($\theta_{\text{max}}$). One tricky aspect about this integral is that the integrand has a singularity at $\theta=\theta_{\text{max}}$.

Darryl Yong 1/10/2020

# 1. First we load in necessary libraries

 We're using quad from scipy.integrate to do the quadrature.

In [22]:
import numpy as np
from scipy import integrate, optimize

# 2. Define some parameters for the problem

- g = gravity (in $m/s^2$)
- thetamax = maximum angle of the swing, measured from vertical (in radians)
- T = time for a complete period of the swing (in seconds)

In [23]:
g = 9.8
thetamax = np.pi / 10
T = 2

# 3. Define the integrand and test out the quad function

In [24]:
integrand = lambda theta: 1/np.sqrt(np.cos(theta)-np.cos(thetamax))

Notice that the integration appears to proceed without a problem even though the integrand has a singularity (divide by zero) at $\theta=\theta_{\text{max}}$.

In [25]:
integrate.quad(integrand, 0, thetamax)

(2.2352224245970485, 8.694200914760586e-11)

To be careful, we should specify that the integrand has a singularity the "points" optional argument.

In [26]:
integrate.quad(integrand, 0, thetamax, points=[thetamax])

(2.2352224245970485, 8.694200914760586e-11)

Here's an alternative to using "points" by specifying a weighting function. This is possible if you know the degree of singularity that is present in the problem. In this case, we know that the integrand goes like $(\theta_{\text{max}}-\theta)^{-1/2}$ as $\theta\to\theta_{\text{max}}$. So, we multiply the integrand by $(\theta_{\text{max}}-\theta)^{1/2}$ to "remove" the singularity. We have to calculate the limit of the new integrand as $\theta\to\theta_{\text{max}}$, which is $2/\sqrt(\sqrt(5-1))$, and then we integrate using a weight function. See https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad. Notice that the estimated error is lower using this method.

In [27]:
integrand2 = lambda theta: 2/np.sqrt(np.sqrt(5)-1) if theta==thetamax else np.sqrt(thetamax-theta)/np.sqrt(np.cos(theta)-np.cos(thetamax))

In [28]:
integrate.quad(integrand2, 0, thetamax, weight='alg', wvar=(0,-1/2))

(2.2352224246011514, 7.226542204582253e-14)

# 4. Solve for length given amplitude and time

We rearrange the equation above to solve for $L$. The answer is shown below, in meters.

In [29]:
ans, err = integrate.quad(integrand, 0, thetamax, points=[thetamax])

In [30]:
ans

2.2352224245970485

In [31]:
L = g * T**2 / (8 * ans)

In [32]:
L

2.1921755732579236